# Schedules Module

This notebook demonstrates the schedule utilities from `specparser.amt.schedules`.

## What is a Schedule?

A **schedule** defines the timing pattern for options strategies - specifically:
- **When to enter** a position (entry date)
- **When to exit** a position (expiry date)  
- **How much weight** to allocate to each component

## Schedule Entry Format

Each schedule consists of one or more entries in the format:

```
{entry}_{expiry}_{weight}
```

For example: `N0_F1_25` means:
- **N0**: Enter at **N**ear month, **0** days offset
- **F1**: Exit at **F**utures expiry, **1**st month
- **25**: This component gets **25%** of the weight

### Entry Codes
| Code | Meaning |
|------|---------|
| `N0`, `N2`, `N5` | Near month + N days offset |
| `F0`, `F10`, `F15` | Far month + N days offset |

### Expiry Codes
| Code | Meaning |
|------|---------|
| `F1`, `F2`, `F3`, `F4` | Futures expiry (1st-4th month) |
| `BDa`, `BDb`, `BDc`, `BDd` | Business day schedules |
| `OVERRIDE` | Uses asset-specific override dates |
| `R2`, `R3` | Roll schedules |

In [ ]:
# Setup: Imports and helper function
import pandas as pd
from specparser.amt import (
    # Schedule loading
    get_schedule,
    find_schedules,
    # Straddle expansion
    get_expand_ym,
    find_straddle_ym,
    get_straddle_yrs,
    find_straddle_yrs,
    # Straddle parsing
    ntr, ntry, ntrm,
    xpr, xpry, xprm,
    ntrc, ntrv, xprc, xprv, wgt,
    # Days computation
    get_days_ym,
    straddle_days,
    count_straddle_days,
    count_straddles_days,
    find_straddle_days,
    # Table utilities
    table_to_rows,
    table_column,
    print_table,
)

# Path to AMT file
AMT_PATH = "../data/amt.yml"

def show(table):
    """Display a table as a pandas DataFrame for pretty rendering."""
    t = table_to_rows(table) if table.get("orientation") == "column" else table
    return pd.DataFrame(t["rows"], columns=t["columns"])

---
## 2. Loading Schedules

### `get_schedule(path, underlying)`

Get the expiry schedule for a single asset by its Underlying value.

In [ ]:
# Get schedule for a single asset
schedule = get_schedule(AMT_PATH, "CL Comdty")
show(schedule)

**Schedule Table Columns:**

| Column | Description |
|--------|-------------|
| `schcnt` | Total number of schedule components |
| `schid` | This component's ID (1 to schcnt) |
| `asset` | The underlying asset |
| `ntrc` | Entry code (N=near, F=far) |
| `ntrv` | Entry value (day offset) |
| `xprc` | Expiry code (F1, BD, OVERRIDE, etc.) |
| `xprv` | Expiry value |
| `wgt` | Weight percentage |

In [ ]:
# Asset without a schedule returns schcnt=0
no_schedule = get_schedule(AMT_PATH, "NONEXISTENT_ASSET")
print(f"Rows: {no_schedule['rows']}")

### `find_schedules(path, pattern, live_only)`

Find schedules for all assets matching a regex pattern.

In [ ]:
# Find schedules for commodities (CL, GC, etc.)
schedules = find_schedules(AMT_PATH, pattern="^(CL|GC|C) ", live_only=True)
show(schedules)

In [ ]:
# Find all live asset schedules
all_schedules = find_schedules(AMT_PATH, pattern=".", live_only=True)
print(f"Total schedule components: {len(all_schedules['rows'])}")

# Show unique asset count
assets = set(row[2] for row in all_schedules["rows"])  # col 2 is 'asset'
print(f"Unique assets with schedules: {len(assets)}")

---
## 3. Understanding Schedule Components

Let's examine a schedule in detail.

In [ ]:
# Get a typical commodity schedule
schedule = get_schedule(AMT_PATH, "CL Comdty")

for row in schedule["rows"]:
    schcnt, schid, asset, entry_code, entry_val, expiry_code, expiry_val, weight = row
    print(f"Component {schid}/{schcnt}:")
    print(f"  Entry: {entry_code}{entry_val} (code={entry_code}, value={entry_val})")
    print(f"  Expiry: {expiry_code}{expiry_val} (code={expiry_code}, value={expiry_val})")
    print(f"  Weight: {weight}%")
    print()

### Interpreting Entry Codes

- **N** (Near): Entry is relative to the near (current) month
- **F** (Far): Entry is relative to the far (next) month

The value is the day offset within that month.

### Interpreting Expiry Codes

| Code | Meaning |
|------|---------|
| `F1`, `F2`, `F3`, `F4` | Exit at 1st/2nd/3rd/4th month futures expiry |
| `OVERRIDE` | Exit at asset-specific override date from CSV |
| `BD` + letter | Business day schedule (a, b, c, d variations) |
| `R2`, `R3` | Roll schedule variants |

---
## 4. Expanding Schedules to Straddles

A **straddle string** is a packed representation that includes:
- Entry year-month
- Expiry year-month  
- The entry/expiry codes and values
- The weight

### `get_expand_ym(path, underlying, year, month)`

Expand a single asset's schedule for a specific year/month.

In [ ]:
# Expand CL Comdty for June 2024
straddles = get_expand_ym(AMT_PATH, "CL Comdty", 2024, 6)
show(straddles)

In [ ]:
# Look at the straddle string format
for row in straddles["rows"]:
    asset, straddle = row
    print(f"Asset: {asset}")
    print(f"Straddle: {straddle}")
    print()

### `find_straddle_ym(path, year, month, pattern, live_only)`

Expand multiple assets for a specific year/month.

In [ ]:
# Expand all equity assets for June 2024
equity_straddles = find_straddle_ym(AMT_PATH, 2024, 6, pattern="Index$|Equity$", live_only=True)
print(f"Found {len(equity_straddles['rows'])} straddle components")
show(equity_straddles)

### `get_straddle_yrs(path, underlying, start_year, end_year)`

Expand a single asset across a year range.

In [ ]:
# Expand CL Comdty for all of 2024
year_straddles = get_straddle_yrs(AMT_PATH, "CL Comdty", 2024, 2024)
print(f"Total straddles: {len(year_straddles['rows'])}")
print(f"(12 months × schedule components)")

# Show first few
show(year_straddles)

### `find_straddle_yrs(path, start_year, end_year, pattern, live_only)`

Expand multiple assets across a year range.

In [ ]:
# Expand all commodities for 2024
commodity_straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern="Comdty$", live_only=True)
print(f"Total commodity straddles for 2024: {len(commodity_straddles['rows'])}")

---
## 5. Straddle String Format

The straddle string packs all schedule information into a single string:

```
|2023-12|2024-01|N|0|OVERRIDE|15|33.3|
 ^^^^^^^ ^^^^^^^ ^ ^  ^^^^^^^ ^^ ^^^^
    |       |    | |     |    |   |
    |       |    | |     |    |   weight (%)
    |       |    | |     |    expiry value
    |       |    | |     expiry code
    |       |    | entry value
    |       |    entry code
    |       expiry (YYYY-MM)
    entry (YYYY-MM)
```

The format is: `|entry_date|expiry_date|ntrc|ntrv|xprc|xprv|wgt|`

In [ ]:
# Example straddle string
example = "|2024-05|2024-06|N|0|F1|0|50|"
print(f"Straddle: {example}")
print()
print("Parsed components:")
print(f"  Entry date:  {example[1:8]}")
print(f"  Expiry date: {example[9:16]}")
print(f"  Components:  {example[17:-1].split('|')}")

---
## 6. Parsing Straddle Strings

The schedules module provides dedicated functions for parsing straddle strings.

### Date Extraction

In [ ]:
straddle = "|2023-12|2024-03|N|5|OVERRIDE|15|33.3|"

# Entry date functions
print("Entry date parsing:")
print(f"  ntr(s)  = {ntr(straddle)!r}")    # Full entry date string
print(f"  ntry(s) = {ntry(straddle)}")      # Entry year (int)
print(f"  ntrm(s) = {ntrm(straddle)}")      # Entry month (int)
print()

# Expiry date functions
print("Expiry date parsing:")
print(f"  xpr(s)  = {xpr(straddle)!r}")    # Full expiry date string
print(f"  xpry(s) = {xpry(straddle)}")      # Expiry year (int)
print(f"  xprm(s) = {xprm(straddle)}")      # Expiry month (int)

### Code and Value Extraction

In [ ]:
straddle = "|2023-12|2024-03|N|5|OVERRIDE|15|33.3|"

# Entry code/value
print("Entry code/value:")
print(f"  ntrc(s) = {ntrc(straddle)!r}")   # Entry code
print(f"  ntrv(s) = {ntrv(straddle)!r}")   # Entry value
print()

# Expiry code/value
print("Expiry code/value:")
print(f"  xprc(s) = {xprc(straddle)!r}")   # Expiry code
print(f"  xprv(s) = {xprv(straddle)!r}")   # Expiry value
print()

# Weight
print("Weight:")
print(f"  wgt(s)  = {wgt(straddle)!r}")

### Parsing Functions Summary

| Function | Returns | Example Output |
|----------|---------|----------------|
| `ntr(s)` | Entry date string | `"2023-12"` |
| `ntry(s)` | Entry year (int) | `2023` |
| `ntrm(s)` | Entry month (int) | `12` |
| `xpr(s)` | Expiry date string | `"2024-03"` |
| `xpry(s)` | Expiry year (int) | `2024` |
| `xprm(s)` | Expiry month (int) | `3` |
| `ntrc(s)` | Entry code | `"N"` |
| `ntrv(s)` | Entry value | `"5"` |
| `xprc(s)` | Expiry code | `"OVERRIDE"` |
| `xprv(s)` | Expiry value | `"15"` |
| `wgt(s)` | Weight | `"33.3"` |

---
## 7. Computing Straddle Days

### `get_days_ym(year, month)`

Get all calendar days in a month (utility function).

In [ ]:
# Get all days in June 2024
days = get_days_ym(2024, 6)
print(f"June 2024 has {len(days)} days")
print(f"First day: {days[0]}")
print(f"Last day: {days[-1]}")

### `straddle_days(straddle)` and `count_straddle_days(straddle)`

Get or count the days spanned by a straddle.

In [ ]:
# A straddle from Jan to Mar 2024
straddle = "|2024-01|2024-03|N|0|F1|0|100|"

# Get all days
days = straddle_days(straddle)
print(f"Straddle spans {len(days)} days")
print(f"From: {days[0]}")
print(f"To: {days[-1]}")
print()

# Count days (faster if you just need the count)
n = count_straddle_days(straddle)
print(f"Day count: {n}")

### `count_straddles_days(straddles)`

Count total days across a table of straddles.

In [ ]:
# Get straddles for an asset
straddles = get_straddle_yrs(AMT_PATH, "CL Comdty", 2024, 2024)

# Total straddle-days
total_days = count_straddles_days(straddles)
print(f"CL Comdty 2024: {len(straddles['rows'])} straddles, {total_days} total straddle-days")

### `find_straddle_days(path, start_year, end_year, pattern, live_only)`

Expand straddles to a day-level table. Returns column-oriented for efficiency.

In [ ]:
# Expand to day-level detail
days_table = find_straddle_days(AMT_PATH, 2024, 2024, pattern="^CL Comdty$", live_only=False)

print(f"Orientation: {days_table['orientation']}")
print(f"Columns: {days_table['columns']}")
print(f"Total rows: {len(days_table['rows'][0])}")  # column-oriented!

# Convert to row-oriented to view
days_rows = table_to_rows(days_table)
show(days_rows)

---
## 8. Practical Examples

### Example 1: Analyze Schedule Patterns by Asset Class

In [ ]:
# Compare schedule complexity across asset classes
patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
]

for pattern, name in patterns:
    schedules = find_schedules(AMT_PATH, pattern, live_only=True)
    if schedules["rows"]:
        # Count unique assets and total components
        assets = set(row[2] for row in schedules["rows"])
        print(f"{name}: {len(assets)} assets, {len(schedules['rows'])} schedule components")
        
        # Average components per asset
        avg = len(schedules["rows"]) / len(assets) if assets else 0
        print(f"  Average components per asset: {avg:.1f}")
        print()

### Example 2: Monthly Straddle Distribution

In [ ]:
# Get all straddles for 2024
straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern=".", live_only=True)

# Count straddles by expiry month
from collections import Counter
months = Counter()
for row in straddles["rows"]:
    straddle = row[1]
    month = xprm(straddle)
    months[month] += 1

print("Straddles by expiry month:")
for month in sorted(months.keys()):
    print(f"  Month {month:2d}: {months[month]:4d} straddles")

### Example 3: Working with Day-Level Data

In [ ]:
# Expand commodities to day-level
days_table = find_straddle_days(AMT_PATH, 2024, 2024, pattern="Comdty$", live_only=True)

# Convert to pandas for analysis
df = pd.DataFrame({
    "asset": days_table["rows"][0],
    "straddle": days_table["rows"][1],
    "date": days_table["rows"][2],
})

print(f"Total rows: {len(df)}")
print(f"\nAssets: {df['asset'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Positions by date (how many straddles are active on each day)
positions_per_day = df.groupby("date").size()
print(f"\nAverage active positions per day: {positions_per_day.mean():.1f}")

---
## 9. Summary

### Schedule Loading Functions

| Function | Description |
|----------|-------------|
| `get_schedule(path, underlying)` | Get schedule for one asset |
| `find_schedules(path, pattern, live_only)` | Find schedules by regex pattern |

### Straddle Expansion Functions

| Function | Description |
|----------|-------------|
| `get_expand_ym(path, underlying, year, month)` | Single asset, single month |
| `find_straddle_ym(path, year, month, pattern, live_only)` | Multiple assets, single month |
| `get_straddle_yrs(path, underlying, start_year, end_year)` | Single asset, year range |
| `find_straddle_yrs(path, start_year, end_year, pattern, live_only)` | Multiple assets, year range |

### Straddle Parsing Functions

| Function | Returns | Description |
|----------|---------|-------------|
| `ntr(s)` | `str` | Entry date (YYYY-MM) |
| `ntry(s)` | `int` | Entry year |
| `ntrm(s)` | `int` | Entry month |
| `xpr(s)` | `str` | Expiry date (YYYY-MM) |
| `xpry(s)` | `int` | Expiry year |
| `xprm(s)` | `int` | Expiry month |
| `ntrc(s)` | `str` | Entry code |
| `ntrv(s)` | `str` | Entry value |
| `xprc(s)` | `str` | Expiry code |
| `xprv(s)` | `str` | Expiry value |
| `wgt(s)` | `str` | Weight |

### Days Computation Functions

| Function | Description |
|----------|-------------|
| `get_days_ym(year, month)` | All days in a month |
| `straddle_days(straddle)` | All days in a straddle period |
| `count_straddle_days(straddle)` | Count days in straddle |
| `count_straddles_days(table)` | Total days in straddles table |
| `find_straddle_days(path, start, end, pattern, live)` | Expand to day-level table |